In [3]:
from keras.layers import Embedding, TimeDistributed, RepeatVector, LSTM, concatenate , Input, Reshape, Dense
# from keras.preprocessing.image import array_to_img, img_to_array, load_img
from tensorflow.keras.utils import array_to_img, img_to_array, load_img
import numpy as np
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model
from IPython.core.display import display, HTML

/tmp/ipykernel_1861/3313140820.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [4]:
#Length of longest sentence
max_caption_len = 3
#Size of vocabulary 
vocab_size = 3

In [5]:
# Load one screenshot for each word, and turn them into digits 
images = []
for i in range(2):
    images.append(img_to_array(load_img('screenshot.jpg', target_size=(224, 224))))
images = np.array(images, dtype=float)
# Preprocess input for the VGG16 model
images = preprocess_input(images)

In [6]:
#Turn start tokens into one-hot encoding
html_input = np.array(
            [[[0., 0., 0.], #start
             [0., 0., 0.],
             [1., 0., 0.]],
             [[0., 0., 0.], #start <HTML>Hello World!</HTML>
             [1., 0., 0.],
             [0., 1., 0.]]])

#Turn next word into one-hot encoding
next_words = np.array(
            [[0., 1., 0.], # <HTML>Hello World!</HTML>
             [0., 0., 1.]]) # end

In [7]:
# Load the VGG16 model trained on imagenet and output the classification feature
VGG = VGG16(weights='imagenet', include_top=True)
# Extract the features from the image
features = VGG.predict(images)

2022-11-14 08:02:44.282506: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-14 08:02:44.282545: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-14 08:02:44.282575: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (emilwallner-screenshott-jtzdjj5biyj): /proc/driver/nvidia/version does not exist
2022-11-14 08:02:44.282781: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


1/1 [==============================] - 1s 1s/step


In [8]:
#Load the feature to the network, apply a dense layer, and repeat the vector
vgg_feature = Input(shape=(1000,))
vgg_feature_dense = Dense(5)(vgg_feature)
vgg_feature_repeat = RepeatVector(max_caption_len)(vgg_feature_dense)
# Extract information from the input seqence 
language_input = Input(shape=(vocab_size, vocab_size))
language_model = LSTM(5, return_sequences=True)(language_input)

In [9]:
# Concatenate the information from the image and the input
decoder = concatenate([vgg_feature_repeat, language_model])
# Extract information from the concatenated output
decoder = LSTM(5, return_sequences=False)(decoder)
# Predict which word comes next
decoder_output = Dense(vocab_size, activation='softmax')(decoder)
# Compile and run the neural network
model = Model(inputs=[vgg_feature, language_input], outputs=decoder_output)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [10]:
# Train the neural network
model.fit([features, html_input], next_words, batch_size=2, shuffle=False, epochs=1000)

Epoch 1/1000
1/1 [==============================] - 3s 3s/step - loss: 1.0957
Epoch 2/1000
1/1 [==============================] - 0s 7ms/step - loss: 1.0858
Epoch 3/1000
1/1 [==============================] - 0s 6ms/step - loss: 1.0786
Epoch 4/1000
1/1 [==============================] - 0s 7ms/step - loss: 1.0725
Epoch 5/1000
1/1 [==============================] - 0s 6ms/step - loss: 1.0669
Epoch 6/1000
1/1 [==============================] - 0s 6ms/step - loss: 1.0617
Epoch 7/1000
1/1 [==============================] - 0s 6ms/step - loss: 1.0568
Epoch 8/1000
1/1 [==============================] - 0s 6ms/step - loss: 1.0521
Epoch 9/1000
1/1 [==============================] - 0s 6ms/step - loss: 1.0476
Epoch 10/1000
1/1 [==============================] - 0s 6ms/step - loss: 1.0431
Epoch 11/1000
1/1 [==============================] - 0s 6ms/step - loss: 1.0388
Epoch 12/1000
1/1 [==============================] - 0s 6ms/step - loss: 1.0346
Epoch 13/1000
1/1 [==============================]

In [11]:
start_token = [1., 0., 0.] # start
sentence = np.zeros((1, 3, 3)) # [[0,0,0], [0,0,0], [0,0,0]]
sentence[0][2] = start_token # place start in empty sentence

In [12]:
# Making the first prediction with the start token
second_word = model.predict([np.array([features[1]]), sentence])

1/1 [==============================] - 1s 622ms/step


In [13]:
# Put the second word in the sentence and make the final prediction
sentence[0][1] = start_token
sentence[0][2] = np.round(second_word)
third_word = model.predict([np.array([features[1]]), sentence])

1/1 [==============================] - 0s 43ms/step


In [14]:
# Place the start token and our two predictions in the sentence 

In [15]:
sentence[0][0] = start_token
sentence[0][1] = np.round(second_word)
sentence[0][2] = np.round(third_word)

In [16]:
# Transform our one-hot predictions into the final tokens
vocabulary = ["start", "<HTML><center><H1>Hello World!</H1><center></HTML>", "end"]
html = ""
for i in sentence[0]:
    html += vocabulary[np.argmax(i)] + ' '

In [17]:
from IPython.core.display import display, HTML
display(HTML(html[6:49]))

/tmp/ipykernel_1861/3207975177.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
